# Implementing decision trees from scratch

In [2]:
import numpy as np
from sklearn.datasets import load_iris

In [35]:
iris_dataset = load_iris()
X = iris_dataset.data[:,:2]
y = iris_dataset.target
Xy = np.column_stack((X,y))

In [36]:
Xy = np.column_stack((X,y))
Xy[:3]

array([[ 5.1,  3.5,  0. ],
       [ 4.9,  3. ,  0. ],
       [ 4.7,  3.2,  0. ]])

In [37]:
def gini_index(groups, classes):
    n_samples = sum([len(group) for group in groups])
    gini = 0
    for group in groups:
        size = float(len(group))
        if size == 0:
            continue
        score = 0     
        for class_value in classes:
            p = [row[-1] for row in group].count(class_value) / size # Calculate gini index for each attribute
            score += p*p
        gini += (1-score)*(size/n_samples)   # Weighted average
    return gini 

In [38]:
# Testing gini_index
z = 40
groups = [Xy[:z, :], Xy[z:,:]]
classes = np.unique(Xy[:,-1])
print(gini_index(groups,classes))

0.42424242424242425


In [39]:
def split(feat, val, Xy):
    Xi_left = np.array([]).reshape(0,3)
    Xi_right = np.array([]).reshape(0,3)
    for i in Xy:
        #print(i.shape)
        if i[feat] <= val:
            Xi_left = np.vstack((Xi_left,i))
        if i[feat] > val:
            Xi_right = np.vstack((Xi_right,i))
    return Xi_left, Xi_right

In [40]:
a, b = split(0, 6.3, Xy)

In [30]:
# Evaluating the cost of splitting at an attribute and selecting the best split (attribute that minimises gini_index)
def get_split(dataset):
    classes = np.unique(dataset[:,-1])
    best_feature = 1000
    best_val = 1000
    best_score = 1000
    best_groups = None
    
    for feature in range(dataset.shape[1]-1):
        for Xi in dataset:
            groups = split(feature, Xi[feature], dataset)
        gini_score = gini_index(groups, classes)
        if gini_score < best_score:
            best_feature = feature
            best_val = Xi[feature]
            best_score = gini_score
            best_groups = groups
    
    output = {}
    output["feature"] = best_feature
    output["val"] = best_val
    output["groups"] = best_groups
    
    return output

In [41]:
split_dictionary = get_split(Xy)

In [42]:
def terminal_node(group):
    classes, counts = np.unique(group[:,-1],return_counts=True)
    return classes[np.argmax(counts)]

In [43]:
z = 40
groups = [Xy[:z,:],Xy[z:,:]]
group = groups[0]
terminal_node(group)

0.0

In [44]:
np.unique?

In [52]:
def split_branch(node, max_depth, min_num_sample, depth):
    left_node, right_node = node['groups']
    del(node['groups'])
    if not isinstance(left_node,np.ndarray) or not isinstance(right_node,np.ndarray):
        node['left'] = node['right'] = terminal_node(left_node + right_node)
        return
    if depth >= max_depth:
        node['left'] = terminal_node(left_node)
        node['right'] = terminal_node(right_node)
        return
    if len(left_node) <= min_num_sample:
        node['left'] = terminal_node(left_node)
    else:
        node['left'] = get_split(left_node)
        split_branch(node['left'], max_depth, min_num_sample, depth+1)
    if len(right_node) <= min_num_sample:
        node['right'] = terminal_node(right_node)
    else:
        node['right'] = get_split(right_node)
        split_branch(node['right'], max_depth, min_num_sample, depth+1)

In [53]:
split_dict = get_split(Xy)
left, right = split_dictionary['groups']

In [54]:
def build_tree(Xy, max_depth, min_num_sample):
    root = get_split(Xy)
    split_branch(root, max_depth, min_num_sample, 1)
    return root

In [55]:
Xy = np.column_stack((X,y))
tree = build_tree(Xy, 2, 30)

In [56]:
np.vstack?